In [90]:
import cvxpy as cp
import numpy as np
import cmath
import hsbalance as hs
import matplotlib.pyplot as plt
%matplotlib widget

# Introduction

In these notes, we are going to discuss the issues related to robust optimization.
a. What is Robust Optimization?
Optimization process is not determinant, this means that we have no clue or determinant check if our solution is the optimum one. Optimization is prone to error if outliers exist. Noisy data and uncertainty readings. Methods, especially least squares method, can be more effected with outliers than others.  
b. Rotor Balancing Robust Optimization:
In our problem of rotor balancing, solution can be greatly effected by outliers for the following reasons:  
1. Measurements are low in numbers: Maximum field measuring points number is still tiny mathematically-wise.
2. Sensor faults: a sensor can go wrong easily for a turbine that runs for 20 years. Calibration is not an easy or a cheap process.  
3. Accumulation of errors: The process of measuring a point in a turbine/compressor usually involves a sensor and long cables in series (vibration cards and junctions). Those produce errors in series and cause the summation to infiltrate.
4. Measuring times: Turbo-Machinery is often connected to large process, which make it start and stop time costly. This encourages for accurate balancing with the fewest number of starts.

# Simulating Data

We generate random data with known solution W

In [112]:
np.random.seed(42)
real = np.random.rand(5,3)
imag = np.random.rand(5,3)
alpha = real + imag * 1j
# Assume W is equal to 10 grams @ 0 degree at all planes
W = [[10], [10], [10]]
# Create intial condtioning column array A with random noise to simulate actual problem
A = - alpha @ W + np.random.normal(1, size=(5,1))

# Create a hsbalance model

In [113]:
model_alpha = hs.Alpha()
model_alpha.add(alpha)
model = hs.LeastSquares(A, model_alpha)
W_model = model.solve()
hs.convert_cart_math(W_model)

array([['9.124@2.2'],
       ['10.11@358.7'],
       ['10.072@1.9']], dtype='<U11')

Calculate the Error

In [116]:
hs.rmse(W_model - W)

0.5115

# Faulty Sensor

To simulate a faulty sensor we will assume that the sensor at the first plane has gone completely fault and gives no output (i.e. equal 0)

In [118]:
A[0] = 0
model_alpha = hs.Alpha()
model_alpha.add(alpha)
model = hs.LeastSquares(A, model_alpha)
W_model = model.solve()
hs.convert_cart_math(W_model)

array([['11.501@17.9'],
       ['6.339@294.5'],
       ['7.434@37.1']], dtype='<U11')

In [119]:
hs.rmse(W_model - W)

6.3579

As we see, the error has gone extremly high and obviously unacceptable.  
If we acknowldge the sensor error, we can use weight least squares method to solve the problem instead.  
This can be done by assign the argument solver='WLS' in solver method of least squares. and pass the coefficient column array C to the model.   
In our case we will give all the sensor the same weight as 1. Only the sensor suspected we will assume a weight of 0. The previous setup will completely discard the sensor from caluclations.

In [120]:
C = np.ones(A.shape)
C[0] = 0
model = hs.LeastSquares(A, model_alpha, C)
W_model = model.solve(solver='WLS')
hs.convert_cart_math(W_model)

array([['9.272@2.6'],
       ['9.699@358.5'],
       ['10.032@3.4']], dtype='<U11')

In [122]:
hs.rmse(W_model - W)

0.6087

We can notice that we restored a low level of error when discarding the faulty sensor.

In [133]:
model = hs.LeastSquares(A, model_alpha)
W_model = model.solve(solver='huber')
hs.convert_cart_math(W_model)

CustomError: Unrecognized Solver name

In [127]:
help(hs.LeastSquares)

Help on class LeastSquares in module hsbalance.model:

class LeastSquares(_Model)
 |  LeastSquares(A: <built-in function array>, alpha: 'instance of Alpha class', C=array([0.]), name='')
 |  
 |  subclass of Model
 |  solving the model using Least squares method, The objective function
 |  is to minimize the least squares of residual vibration.
 |  
 |  Method resolution order:
 |      LeastSquares
 |      _Model
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, A: <built-in function array>, alpha: 'instance of Alpha class', C=array([0.]), name='')
 |      Instantiate the model
 |      Args:
 |      A: Initial vibration vector -> np.ndarray
 |      ALPHA: Influence coefficient matrix -> class Alpha
 |      C: Weighted Least squares coefficients
 |      name: optional name of the model -> string
 |  
 |  solve(self, solver='OLE')
 |      Method to solve the model
 |      Args:
 |          solver:'OLE' Ordinary Least Squares method
 |          'Huber': Uses H